# Interactive Computing with Dynamic Compute - ImageStacks
__________________

In the [previous notebook](./01%20Interactive%20Computing%20with%20Mosaics.ipynb) we were introduced to interactive mapping with mosaics. Here we will explore the [`ImageStack`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.ImageStack) and its expanded functionality to analyze, filter, and manipulate aggregated stacks of raster imagery. 

For a more detailed look at all its classes and their available methods please visit the [`Dynamic Compute`](https://docs.descarteslabs.com/api/dynamic-compute.html) Documentation page.

In [ ]:
import descarteslabs.dynamic_compute as dc
from descarteslabs.dynamic_compute import ImageStack, Mosaic

We'll start by defining an interactive viewport over the Austin, Texas metro area and add a mosaic of RGB data:

In [ ]:
m = dc.map
m.center = 30.2743226, -97.7387934
m.zoom = 13

In [ ]:
s2_mosaic = Mosaic.from_product_bands(
    "esa:sentinel-2:l2a:v1",
    "red green blue",
    start_datetime="2023-01-01",
    end_datetime="2023-04-01",
)
s2_mosaic.pick_bands("red green blue").visualize("S2 TCC", m)

In [ ]:
m

## ImageStacks
The first thing we notice in our map viewport is that it's full of clouds! Let's address that by first creating an [`ImageStack.from_product_bands()`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.ImageStack.from_product_bands), with the same arguments as before:

In [ ]:
s2_stack = dc.ImageStack.from_product_bands(
    "esa:sentinel-2:l2a:v1",
    "red green blue nir swir1",
    start_datetime="2023-01-01",
    end_datetime="2023-04-01",
)
type(s2_stack)

**_Note_**: If you are coming from the legacy Workflows client, the ImageStack is analogous to an ImageCollection with full temporal aggregation capability.

## Filtering ImageStacks
One limitation of a mosaic is that the _aggregation component of our analysis is not present_. With a stack of data we can call [`ImageStack.filter()`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.ImageStack.filter) based off arbitrary metadata properties, such as __cloud_fraction__:

In [ ]:
try:
    # this is the >=1.3.0 dynamic-compute version
    import descarteslabs as dl
    s2_stack_cloudfree = s2_stack.filter(dl.catalog.properties.cloud_fraction < 0.2)
except Exception:
    # this is the <1.3.0 dynamic-compute version
    s2_stack_cloudfree = s2_stack.filter(lambda x: x.cloud_fraction < 0.2)
    
type(s2_stack_cloudfree)

## Visualizing ImageStacks

If we tried calling visualize on this now we would not return any imagery, that is because we need to _reduce_ our data, in this case across it's temporal dimension. 

In the next cell we will call [`ImageStack.median()`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.ImageStack.median) on our filtered data before visualizing onto our map:

In [ ]:
(
    s2_stack_cloudfree.median(axis="images")
    .pick_bands("red green blue")
    .visualize("S2 TCC Cloudfree", m)
)

**Note**: _There are several other operators you can use (e.g. [`min`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.ImageStack.min), [`mean`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.ImageStack.mean), [`max`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.ImageStack.max), [`std`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.ImageStack.std), etc.)_

In [ ]:
m

No more clouds!

## Band Ratios with ImageStacks

As we've seen, the enhanced flexibility afforded when working with image stacks comes with the added complexity of needing to reduce dimensions. 

Here we will calculate the median [_Normalized Difference in Built-up Index_](https://d-nb.info/1195147821/34) of our cloud-free study area:

$NDBI = (SWIR1 - NIR) / (SWIR1 + NIR)$

In [ ]:
nir, swir = s2_stack_cloudfree.unpack_bands("nir swir1")
ndbi = (swir - nir) / (swir + nir)
ndbi.median(axis="images").visualize("NDBI", m)

In [ ]:
m